In [ ]:
import time
import os
import argparse
import cProfile
import pstats
import logging

import numpy as np
import astropy.io.fits as pyfits

from corgihowfsc.utils.howfsc_initialization import get_args_jupyter, load_files
from corgihowfsc.sensing.DefaultProbes import DefaultProbes

import eetc
from eetc.cgi_eetc import CGIEETC

import howfsc
from howfsc.control.cs import ControlStrategy
from howfsc.control.calcjtwj import JTWJMap

from howfsc.model.mode import CoronagraphMode

from howfsc.util.loadyaml import loadyaml
from howfsc.util.gitl_tools import param_order_to_list

from howfsc.gitl import howfsc_computation
from howfsc.precomp import howfsc_precomputation

import cgisim

from howfsc.scripts.gitlframes import sim_gitlframe

eetc_path = os.path.dirname(os.path.abspath(eetc.__file__))
howfscpath = os.path.dirname(os.path.abspath(howfsc.__file__))


defjacpath = os.path.join(os.path.dirname(howfscpath), 'jacdata')

import matplotlib.pyplot as plt
from corgihowfsc.utils.corgisim_gitl_frames import GitlImage

#First, import the module:
import roman_preflight_proper
### Then, run the following command to copy the default prescription file 
roman_preflight_proper.copy_here()

Loading arguments 

In [ ]:
# Load arguments
args = get_args_jupyter(jacpath=defjacpath)

In [ ]:
# Initialise variables etc
otherlist = []
abs_dm1list = []
abs_dm2list = []
framelistlist = []
scalelistout = []
camlist = []

# User params
niter = args.niter
mode = args.mode
isprof = args.profile
logfile = args.logfile
fracbadpix = args.fracbadpix
nbadpacket = args.nbadpacket
nbadframe = args.nbadframe
fileout = args.fileout
stellar_vmag = args.stellarvmag
stellar_type = args.stellartype
stellar_vmag_target = args.stellarvmagtarget
stellar_type_target = args.stellartypetarget
jacpath = args.jacpath
    
modelpath, cfgfile, jacfile, cstratfile, probe0file, probe1file, probe2file, hconffile, n2clistfiles = load_files(args, howfscpath)

# cfg
cfg = CoronagraphMode(cfgfile)
cstrat = ControlStrategy(cstratfile)


initialising the probes

In [ ]:
# Initialize default probes class
probes = DefaultProbes('default')

# Get DM lists
probefiles = {}
probefiles[0] = probe0file
probefiles[2] = probe1file
probefiles[1] = probe2file

dm1_list, dm2_list = probes.get_dm_probes(cfg, probefiles,
                      scalelist=[0.3, 0.3, 0.3, -0.3, -0.3, -0.3])

# Get probe amplitude lists (this step is slow)
plist, other = probes.get_probe_ap(cfg, dm1_list, dm2_list)

In [ ]:
exptime = 30
gain = 5000
dm1v = dm1_list[1]
dm2v = dm2_list[1]
cor = 'hlc'
crop = (0, 0, 500, 500)

In [ ]:
polaxis=10
# without any dm and ccd parameters (uses defaults, whatever they are for now)

star_spectrum='a5v'
vmag=2.56

# GitlImage using corgisim

In [ ]:
name = 'corgihowfsc'
bandpass = '1'
polaxis = 10

imager = GitlImage(name, cor, bandpass, polaxis, is_noise_free=True, output_dim=51)
exptime = 30
gain = 5000
dm1v = dm1_list[1]
dm2v = dm2_list[1]
crop = (0, 0, 500, 500)
peakflux = 1e6

lind = 0 # getting to the first sub-band

gitlframe = imager.get_image(dm1v, dm2v, exptime, gain, crop, lind,
    cleanrow=1024, cleancol=1024)


In [ ]:
plt.imshow(gitlframe)
plt.colorbar()

In [ ]:
# If one must check with cgisim directly

# image, counts = cgisim.rcgisim(
#     mode, cor, bandpass, polaxis, param_dict,
#     star_spectrum=star_spectrum,
#     star_vmag=vmag
#     )

when imager is 'cgi-howfsc' 

In [ ]:
fixedbp = np.zeros((1024,1024), dtype=bool)
peakflux = 4.72154e+06
exptime = 46.9043

nrow = 153
ncol = 153
lrow = 436
lcol = 436

croplist = (lrow, lcol, nrow, ncol)


In [ ]:
name = 'cgi-howfsc'
imager = GitlImage(name, mode, bandpass, cfg=cfg, cstrat=cstrat, is_noise_free=True)

In [ ]:
lind = 2
gitlframe = imager.get_image(dm1v, dm2v, exptime, gain, croplist, lind, peakflux, cleanrow=1024, cleancol=1024)

In [ ]:
plt.imshow(gitlframe)